##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Using TensorBoard in Notebooks

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

TensorBoard can be used directly within notebook experiences such as [Colab](https://colab.research.google.com/) and [Jupyter](https://jupyter.org/). This can be helpful for sharing results, integrating TensorBoard into existing workflows, and using TensorBoard without installing anything locally.

## Setup

Start by installing TF 2.0 and loading the TensorBoard notebook extension:

**For Jupyter users:** If you’ve installed Jupyter and TensorBoard into
the same virtualenv, then you should be good to go. If you’re using a
more complicated setup, like a global Jupyter installation and kernels
for different Conda/virtualenv environments, then you must ensure that
the `tensorboard` binary is on your `PATH` inside the Jupyter notebook
context. One way to do this is to modify the `kernel_spec` to prepend
the environment’s `bin` directory to `PATH`, [as described here][1].

[1]: https://github.com/ipython/ipykernel/issues/395#issuecomment-479787997


In case you are running a [Docker](https://docs.docker.com/install/) image of [Jupyter Notebook server using TensorFlow's nightly](https://www.tensorflow.org/install/docker#examples_using_cpu-only_images), it is necessary to expose not only the notebook's port, but the TensorBoard's port.

Thus, run the container with the following command:

```
docker run -it -p 8888:8888 -p 6006:6006 \
tensorflow/tensorflow:nightly-py3-jupyter 
```

where the `-p 6006` is the default port of TensorBoard. This will allocate a port for you to run one TensorBoard instance. To have concurrent instances, it is necessary to allocate more ports.


In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

Import TensorFlow, datetime, and os:

In [0]:
import tensorflow as tf
import datetime, os

## TensorBoard in notebooks

Download the [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist) dataset and scale it:

In [0]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Create a very simple model:

In [0]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

Train the model using Keras and the TensorBoard callback:

In [6]:
def train_model():
  
  model = create_model()
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(x=x_train, 
            y=y_train, 
            epochs=5, 
            validation_data=(x_test, y_test), 
            callbacks=[tensorboard_callback])

train_model()

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 11s 180us/sample - loss: 0.4970 - accuracy: 0.8227 - val_loss: 0.4041 - val_accuracy: 0.8541
Epoch 2/5
60000/60000 [==============================] - 11s 178us/sample - loss: 0.3802 - accuracy: 0.8598 - val_loss: 0.3841 - val_accuracy: 0.8624
Epoch 3/5
60000/60000 [==============================] - 10s 174us/sample - loss: 0.3473 - accuracy: 0.8709 - val_loss: 0.3607 - val_accuracy: 0.8676
Epoch 4/5
60000/60000 [==============================] - 10s 164us/sample - loss: 0.3294 - accuracy: 0.8785 - val_loss: 0.3537 - val_accuracy: 0.8704
Epoch 5/5
60000/60000 [==============================] - 10s 168us/sample - loss: 0.3113 - accuracy: 0.8850 - val_loss: 0.3436 - val_accuracy: 0.8782


Start TensorBoard within the notebook using [magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html):

In [7]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 380), started 1:04:03 ago. (Use '!kill 380' to kill it.)

<img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard.png?raw=1"/>

You can now view dashboards such as scalars, graphs, histograms, and others. Some dashboards are not available yet in Colab (such as the profile plugin).

The `%tensorboard` magic has exactly the same format as the TensorBoard command line invocation, but with a `%`-sign in front of it.

You can also start TensorBoard before training to monitor it in progress:

In [8]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 380), started 1:04:03 ago. (Use '!kill 380' to kill it.)

<img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard_two_runs.png?raw=1"/>

The same TensorBoard backend is reused by issuing the same command. If a different logs directory was chosen, a new instance of TensorBoard would be opened. Ports are managed automatically. 

Start training a new model and watch TensorBoard update automatically every 30 seconds or refresh it with the button on the top right:

In [9]:
train_model()

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 10s 173us/sample - loss: 0.4956 - accuracy: 0.8238 - val_loss: 0.4242 - val_accuracy: 0.8502
Epoch 2/5
60000/60000 [==============================] - 10s 163us/sample - loss: 0.3826 - accuracy: 0.8596 - val_loss: 0.4267 - val_accuracy: 0.8427
Epoch 3/5
60000/60000 [==============================] - 10s 164us/sample - loss: 0.3504 - accuracy: 0.8705 - val_loss: 0.3671 - val_accuracy: 0.8676
Epoch 4/5
60000/60000 [==============================] - 10s 166us/sample - loss: 0.3295 - accuracy: 0.8770 - val_loss: 0.3636 - val_accuracy: 0.8697
Epoch 5/5
60000/60000 [==============================] - 10s 166us/sample - loss: 0.3140 - accuracy: 0.8827 - val_loss: 0.3484 - val_accuracy: 0.8706


You can use the `tensorboard.notebook` APIs for a bit more control:

In [10]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

Known TensorBoard instances:
  - port 6006: logdir logs (started 1:04:53 ago; pid 380)


In [11]:
# Control TensorBoard display. If no port is provided, 
# the most recently launched TensorBoard is used
notebook.display(port=6006, height=1000) 

Selecting TensorBoard with logdir logs (started 1:04:53 ago; port 6006, pid 380).


<img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard_tall.png?raw=1"/>